In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
spark = SparkSession.builder \
    .config('spark.driver.memory', '3g') \
    .enableHiveSupport().getOrCreate()

In [6]:
! ls /data

Badges	Comments  PostLinks  Posts  Tags  Users  Votes


In [9]:
base_path = 'file:///data'

In [33]:
posts = spark.read.format('parquet').load(f'{base_path}/Posts/part-00500*')

In [34]:
posts.count()

71561

In [14]:
posts.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



In [15]:
posts.groupBy('PostTypeId').count().show()

+----------+--------+
|PostTypeId|   count|
+----------+--------+
|         1|18154493|
|         6|     324|
|         3|     167|
|         5|   49909|
|         4|   49909|
|         8|       2|
|         7|       4|
|         2|27665009|
+----------+--------+



In [16]:
QUESTION_POST_TYPE = 1
questions = posts.where(col('PostTypeId') == QUESTION_POST_TYPE)

In [18]:
questions.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



### 1. Czy pytanie może być community-owned? (`CommunityOwnedDate`)

In [20]:
questions.where(col('CommunityOwnedDate').isNotNull()).select('Id').first()

Row(Id=839899)

### 2. Przykładowy ID zamkniętego pytania z września 2019 (`ClosedDate`)

https://stackoverflow.com/q/839899

In [22]:
questions.where(year('ClosedDate') == 2019) \
    .where(month('ClosedDate') == 9).select('Id').show()

+--------+
|      Id|
+--------+
|  206258|
| 2279993|
|23043938|
|53167196|
|57614714|
|57649465|
|57689162|
|57029989|
|57576987|
|47334139|
|40689858|
|57722308|
|57726456|
|57727515|
|57728075|
|57730003|
|57733132|
|57734725|
|57734751|
|57736739|
+--------+
only showing top 20 rows



In [24]:

# mało badży (do tego czasu)
# mało tagów
# dużo tagów
# za krótkie pytanie -> mniej niż 100 znaków
# długość pytania
# długość tytułu
# wiek
# autor dopiero zarejestrowany (+1)
# niski view count
# tytuł kończy się znakiem zapytania
# tytuł krótszy niż 5 znaków
# CAPS lock w tytule
# dużo znaków interpunkcyjnych w tytule
# obecność obrazków
# "internet"
# "please"
# title zaczyna się wielką literą
# title zaczyna się od "how", "what" albo "why"
# godzina
# czy weekend?
# ile postów usera zostało zamkniętych (do tego czasu)
# ile pytań zadał dotąd
# ma "defaultowy" nick?
# wpisana lokacja?
# owner user id jest pusty?

## Cechy na początek:

 * ocena na minusie (+1)
 * obecność kodu (+1)
 * autor dopiero zarejestrowany (+1)

In [ ]:
```
this is my code
``` 


-> <pre><code>


this is `my` code

--> <code>

In [49]:
questions \
    .show()

+----+--------+---------------------------------+
|  Id|      Id|how_many_days_ago_user_registered|
+----+--------+---------------------------------+
| 148|   58024|                               41|
| 148|   58425|                               41|
| 148|  143681|                               56|
| 148| 7991542|                             1188|
| 463|  783155|                              260|
| 463|    5606|                                2|
| 463| 1304591|                              379|
| 463| 1521599|                              425|
| 463|  621266|                              213|
| 463| 2047194|                              524|
| 463| 3007533|                              672|
| 463|12776625|                             1524|
| 463|14781537|                             1647|
|1088|  885819|                              281|
|1088|  925981|                              290|
|1088|  829623|                              267|
|1088|    8939|                                0|


In [81]:
import re
def how_many_code_letters(body):
    if body == None:
        return 0 ## or None?
    letters = 0
    for code in re.finditer('<code>([^<]+)</code>', body):
        letters += len(code.group(1))
    return letters

In [82]:
how_many_code_letters('<code>abc</code> <code>1</code>')

4

In [51]:
days_difference_between_reg_and_post = datediff(questions.CreationDate, users.CreationDate)
questions.join(broadcast(users), questions.OwnerUserId == users.Id) \
    .select(
        questions.Id,
        (col('Score')<0).alias('is_score_less_than_zero'),
        'Score',
        col('Body').contains('<code>').alias('has_code_attached'),
        lit(-1).alias('code_length'),
        (days_difference_between_reg_and_post < 2).alias('is_author_just_registered'),
        days_difference_between_reg_and_post.alias('how_many_days_ago_user_registered'),
        col('ClosedDate').isNotNull().alias('is_closed'),
    ).show()

+------+-----------------------+-----+-----------------+-----------+-------------------------+---------------------------------+---------+
|    Id|is_score_less_than_zero|Score|has_code_attached|code_length|is_author_just_registered|how_many_days_ago_user_registered|is_closed|
+------+-----------------------+-----+-----------------+-----------+-------------------------+---------------------------------+---------+
|839854|                  false|    3|             true|         -1|                    false|                               30|    false|
|839856|                  false|   13|            false|         -1|                     true|                                0|    false|
|839861|                  false|    2|            false|         -1|                    false|                              204|    false|
|839873|                  false|    1|            false|         -1|                    false|                               95|    false|
|839874|                  f

## Przydatne funkcje

 * `unix_timestamp`
 * `regexp_extract`

In [29]:
users = spark.read.parquet(f'{base_path}/Users')
users.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Reputation: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- AboutMe: string (nullable = true)
 |-- Views: integer (nullable = true)
 |-- UpVotes: integer (nullable = true)
 |-- DownVotes: integer (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- AccountId: integer (nullable = true)

